In [25]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [26]:
# Завантаження та підготовка даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train_resized = np.array([np.pad(image, ((2, 2), (2, 2)), mode='constant') for image in x_train])
x_test_resized = np.array([np.pad(image, ((2, 2), (2, 2)), mode='constant') for image in x_test])
x_train_resized = np.stack((x_train_resized,) * 3, axis=-1)
x_test_resized = np.stack((x_test_resized,) * 3, axis=-1)
x_train_resized = x_train_resized.astype('float32') / 255.0
x_test_resized = x_test_resized.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [27]:
# Завантаження моделі VGG16 без верхніх шарів
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Заморожуємо ваги згорткової бази VGG16
vgg_base.trainable = False

In [28]:
# Побудова моделі
# Побудова моделі
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

In [29]:
# Компіляція моделі
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
# Навчання моделі
model.fit(x_train_resized, y_train, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 309s 729ms/step - loss: 0.6305 - accuracy: 0.7805 - val_loss: 0.4783 - val_accuracy: 0.8302
Epoch 2/10
422/422 [==============================] - 941s 2s/step - loss: 0.4389 - accuracy: 0.8413 - val_loss: 0.4438 - val_accuracy: 0.8398
Epoch 3/10
422/422 [==============================] - 1070s 3s/step - loss: 0.3993 - accuracy: 0.8538 - val_loss: 0.4162 - val_accuracy: 0.8515
Epoch 4/10
422/422 [==============================] - 1044s 2s/step - loss: 0.3733 - accuracy: 0.8623 - val_loss: 0.3778 - val_accuracy: 0.8635
Epoch 5/10
422/422 [==============================] - 1111s 3s/step - loss: 0.3583 - accuracy: 0.8685 - val_loss: 0.3812 - val_accuracy: 0.8625
Epoch 6/10
422/422 [==============================] - 1000s 2s/step - loss: 0.3436 - accuracy: 0.8734 - val_loss: 0.3751 - val_accuracy: 0.8613
Epoch 7/10
422/422 [==============================] - 1138s 3s/step - loss: 0.3318 - accuracy: 0.8781 - val_loss: 0.3659 - val_accuracy

In [31]:
# Оцінка точності на тестових даних
test_loss, test_acc = model.evaluate(x_test_resized, y_test)
print("Test Accuracy:", test_acc)

313/313 [==============================] - 169s 536ms/step - loss: 0.3756 - accuracy: 0.8655
Test Accuracy: 0.8654999732971191
